# Adalflow RAG Playbook example

There are different patterns to build a RAG:

- RAG with separate data process pipeline and a RAG task pipeline. This fits into a scenario where there is lots of data in production database, and we preprocess the data to embeddings and then we build a RAG task pipeline that retrieves context in multiple stages.

- RAG with dynamic data access and caching the embedding dynamically in a local storage.

Here we will have have a look at an example with a local DB using FAISS

In [1]:
from IPython.display import clear_output

!pip install -U adalflow[openai,groq,faiss-cpu]

clear_output()


In [2]:
import os
from getpass import getpass

# Prompt user to enter their API keys securely
openai_api_key = getpass("Please enter your OpenAI API key: ")
groq_api_key = getpass("Please enter your GROQ API key: ")

# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['GROQ_API_KEY'] = groq_api_key

print("API keys have been set.")


Please enter your OpenAI API key: ··········
Please enter your GROQ API key: ··········
API keys have been set.


## Design

Some libraries may use hooks [2] and callbacks [3] [4], or advanced web-based debugging tools [5] [6] [7]. Hooks and callbacks are conceptually similar in that they both allow users to execute custom code at specific points during the execution of a program. Both provide mechanisms to inject additional behavior in response to certain events or conditions, without modifying the core logic. PyTorch defines, registers, and executes hooks mainly in its base classes like nn.Module and Tensor, without polluting the functional and user-facing APIs.

At this point, our objectives are:

1. Maximize debugging capabilities via the simple logging module to keep the source code clean.

2. Additionally, as we can’t always control the outputs of generators, we will provide customized logger and tracers(drop-in decorators) for them, for which we will explain in Tracing. This will not break the first objective.

In the future, when we have more complex requirements from users, we will consider adding hooks/callbacks but we will do it in a way to keep the functional and user-facing APIs clean.

In [3]:
import logging

log = logging.getLogger(__name__)

In [4]:
from adalflow.utils.logger import get_logger


root_logger = get_logger()

In [5]:
from adalflow.utils.logger import printc

printc("All logging examples are done. Feeling green!", color="green")

2024-11-28 13:39:41 - [<ipython-input-5-9bd1accb40e0>:3:<cell line: 3>] - All logging examples are done. Feeling green!


Set up all logs in one file

Assume your source code is at src/task.py. You can log simply by:

In [6]:
import logging

log = logging.getLogger(__name__)

class Task:
    def __init__(self):
        log.info("This is a user program child logger")

In [7]:
import logging
from adalflow.utils.logger import get_logger

root_logger = get_logger(level="DEBUG", save_dir="./logs") # log to ./logs/lib.log

# run code from the library components such as generator
# ....

root_logger.info("This is the log in the main file")

2024-11-28 13:39:46 - <ipython-input-7-f4c1161cc964> - INFO - [<ipython-input-7-f4c1161cc964>:9:<cell line: 9>] - This is the log in the main file


Separate library and application logs

In [8]:
from adalflow.utils.logger import get_logger

app_logger = get_logger(name="my_app", level="DEBUG", save_dir="./logs") # log to ./logs/my_app.log

class Task:
    def __init__(self):
        app_logger.info("This is a user program child logger")